In [ ]:
# @title
from IPython.display import display, HTML

display(HTML("""
<script>
const firstCell = document.querySelector('.cell.code_cell');
if (firstCell) {
  firstCell.querySelector('.input').style.pointerEvents = 'none';
  firstCell.querySelector('.input').style.opacity = '0.5';
}
</script>
"""))

html = """
<div style="display:flex; flex-direction:column; align-items:center; text-align:center; gap:12px; padding:8px;">
  <h1 style="margin:0;">👋 Welcome to <span style="color:#1E88E5;">Algopath Coding Academy</span>!</h1>

  <img src="https://raw.githubusercontent.com/sshariqali/mnist_pretrained_model/main/algopath_logo.jpg"
       alt="Algopath Coding Academy Logo"
       width="400"
       style="border-radius:15px; box-shadow:0 4px 12px rgba(0,0,0,0.2); max-width:100%; height:auto;" />

  <p style="font-size:16px; margin:0;">
    <em>Empowering young minds to think creatively, code intelligently, and build the future with AI.</em>
  </p>
</div>
"""

display(HTML(html))

## Day 9 — Task 1: Bigram Name Generator 🟢

---

### 🎯 **Goal**

Build a **character-level bigram language model** that generates novel human names — following the same architecture from Day 7 (Parts 1 & 2) but applied to a **completely new domain**: baby names!

Instead of generating Shakespeare-like text, our model will learn the statistical patterns of real names and invent plausible new ones.

---

### 📋 **Agenda**

| Section | Topic | Description |
|:-------:|-------|-------------|
| 1 | **Imports & Hyperparameters** | Load PyTorch and configure training settings |
| 2 | **Loading the Names Dataset** | Download 32K baby names from Karpathy's repo |
| 3 | **Exploring the Data** | Understand name structure, characters, and vocabulary |
| 4 | **Building a Tokenizer** | Create `stoi` / `itos` mappings with a special start/end token |
| 5 | **Data Preparation** | Build bigram (input, target) pairs and batch them |
| 6 | **Building the Bigram Model** | `nn.Embedding`-based bigram language model |
| 7 | **Training** | Train with `CrossEntropyLoss` and track train/val loss |
| 8 | **Generating Names** | Sample plausible new names from the trained model |
| 9 | **Analysis & Visualisation** | Visualise the learned bigram probabilities |

---

### 🎓 **Learning Objectives**

By the end of this notebook, you will:
- ✅ Apply the bigram language model architecture to a new domain
- ✅ Build a character-level tokenizer with a special boundary token (`.`)
- ✅ Prepare name data as bigram input-target pairs
- ✅ Train the model and monitor train/validation loss
- ✅ Generate novel human names by sampling from the model
- ✅ Visualise the learned character transition probabilities

Let's generate some names! 🚀

---
## Section 1: Imports & Hyperparameters

We import PyTorch and **matplotlib** for visualisation. We also define all hyperparameters at the top — a best practice we picked up in Day 7 Part 2.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random

In [ ]:
# ── Hyperparameters ──────────────────────────────────────────
batch_size = 64        # number of bigram pairs processed in parallel
max_iters = 10000     # total training steps
eval_interval = 1000     # how often to print train/val loss
learning_rate = 1e-2     # step size for AdamW optimiser
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200       # batches to average when estimating loss
torch.manual_seed(42)

print(f"Using device: {device}")

---
## Section 2: Loading the Names Dataset

We'll use **Andrej Karpathy's `names.txt`** — a list of 32,033 baby names sourced from the US Social Security Administration (2018). Each line contains one lowercase name.

The file is hosted on GitHub, so we download it directly with `urllib`.

In [ ]:
import urllib.request
import os

url = "https://raw.githubusercontent.com/karpathy/makemore/master/names.txt"
filename = "names.txt"

# Download the file only if it doesn't already exist
if not os.path.exists(filename):
    urllib.request.urlretrieve(url, filename)
    print(f"Downloaded {filename}")
else:
    print(f"{filename} already exists — skipping download")

# Read all names into a list
with open(filename, "r", encoding="utf-8") as f:
    names = f.read().splitlines()

print(f"Total names: {len(names)}")
print(f"First 10 names: {names[:10]}")
print(f"Last 10 names:  {names[-10:]}")

---
## Section 3: Exploring the Data

Before building a model, let's understand our data:
- How long are the names?
- What characters appear?
- What does the distribution of name lengths look like?

In [ ]:
# Basic statistics
lengths = [len(name) for name in names]
print(f"Shortest name: {min(lengths)} characters  (e.g. {[n for n in names if len(n) == min(lengths)][:5]})")
print(f"Longest name:  {max(lengths)} characters  (e.g. {[n for n in names if len(n) == max(lengths)][:3]})")
print(f"Average length: {sum(lengths)/len(lengths):.1f} characters")

In [ ]:
# Distribution of name lengths
plt.figure(figsize=(10, 4))
plt.hist(lengths, bins=range(1, max(lengths)+2), edgecolor='black', alpha=0.7, color='#1E88E5')
plt.xlabel('Name Length (characters)')
plt.ylabel('Count')
plt.title('Distribution of Name Lengths')
plt.xticks(range(1, max(lengths)+1))
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# What characters appear in the dataset?
all_chars = sorted(list(set(''.join(names))))
print(f"Unique characters ({len(all_chars)}): {''.join(all_chars)}")

**Observation:** The dataset contains only lowercase letters (a-z). No digits, spaces, or special characters — perfect for a simple character-level model!

---
## Section 4: Building a Tokenizer (`stoi` / `itos`)

### 🔤 Character-Level Tokenizer

Just like in Day 7, we need to convert characters to integers and back.

**But there's a twist!** In the Shakespeare dataset, we modelled a continuous stream of text. For names, each name is a **separate sequence** with a clear beginning and end.

We introduce a **special boundary token `.`** (dot) to mark the start and end of each name:

```
"emma"  →  .  e  m  m  a  .
```

This way:
- The model learns which characters are likely to **start** a name (after `.`)
- The model learns which characters are likely to **end** a name (before `.`)
- When generating, we start with `.` and stop when the model produces another `.`

The `.` token gets index **0** in our vocabulary.

In [ ]:
# Build the vocabulary: '.' (boundary) + all 26 lowercase letters
chars = ['.'] + all_chars
vocab_size = len(chars)

print(f"Vocabulary ({vocab_size} tokens): {''.join(chars)}")

# Create the mappings
stoi = {ch: i for i, ch in enumerate(chars)}   # string-to-index
itos = {i: ch for i, ch in enumerate(chars)}   # index-to-string

print(f"\nstoi: {stoi}")
print(f"\nitos: {itos}")

In [ ]:
# Test the tokenizer
test_name = "emma"
encoded = [stoi[c] for c in test_name]
decoded = ''.join([itos[i] for i in encoded])
print(f"'{test_name}' → encoded: {encoded} → decoded: '{decoded}'")

---
## Section 5: Data Preparation — Building Bigram Pairs

### What Are Bigrams?

A **bigram** is a pair of consecutive characters. For the name `"emma"` (with boundary tokens), the bigrams are:

| Input (current char) | Target (next char) |
|:-------------------:|:-----------------:|
| `.` | `e` |
| `e` | `m` |
| `m` | `m` |
| `m` | `a` |
| `a` | `.` |

The model's job: **given the current character, predict the next character**.

We build these pairs for ALL names in the dataset.

In [ ]:
# Build all bigram (input, target) pairs from the dataset
xs = []  # inputs
ys = []  # targets

for name in names:
    # Add boundary tokens: .emma.
    chars_in_name = ['.'] + list(name) + ['.']
    for ch1, ch2 in zip(chars_in_name, chars_in_name[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs) 
ys = torch.tensor(ys)

print(f"Total bigram pairs: {len(xs)}")
print(f"\nFirst 10 pairs:")
for i in range(10):
    print(f"  {itos[xs[i].item()]} → {itos[ys[i].item()]}  (indices: {xs[i].item()} → {ys[i].item()})")

### Train / Validation Split

We shuffle and split the data 90/10, just like in Day 7.

In [ ]:
# Shuffle the data
torch.manual_seed(42)
perm = torch.randperm(len(xs))
xs = xs[perm]
ys = ys[perm]

# 90% train, 10% validation
n = int(0.9 * len(xs))
x_train, y_train = xs[:n], ys[:n]
x_val,   y_val   = xs[n:], ys[n:]

print(f"Training pairs:   {len(x_train)}")
print(f"Validation pairs: {len(x_val)}")

### Batching

We sample random mini-batches from the training or validation set. Each sample is a single bigram pair (one input character, one target character).

In [ ]:
def get_batch(split):
    """Sample a random mini-batch of bigram pairs."""
    if split == 'train':
        x_data, y_data = x_train, y_train
    else:
        x_data, y_data = x_val, y_val
    ix = torch.randint(len(x_data), (batch_size,))
    x = x_data[ix].to(device)
    y = y_data[ix].to(device)
    return x, y

# Quick test
xb, yb = get_batch('train')
print(f"Batch input shape:  {xb.shape}")
print(f"Batch target shape: {yb.shape}")
print(f"\nSample pairs from batch:")
for i in range(5):
    print(f"  {itos[xb[i].item()]} → {itos[yb[i].item()]}")

---
## Section 6: Loss Estimation

As we learned in Day 7 Part 2, a single batch loss is noisy. We average over many batches for a stable estimate.

`@torch.no_grad()` disables gradient computation — we're only evaluating, not training.

In [ ]:
@torch.no_grad()
def estimate_loss():
    """Average loss over several batches for both train and val splits."""
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

---
## Section 7: Building the Bigram Model

### Architecture Recap

This is the **same Bigram model** from Day 7, adapted for name generation:

- A single `nn.Embedding(vocab_size, vocab_size)` layer
- Each row `i` of the embedding table stores the **logits** (prediction scores) for "what character comes after character `i`?"
- No hidden layers, no attention — just a direct lookup table

**Key difference from Day 7:**  
- Input is a **single character** (not a sequence), so the forward pass is simpler
- We add a `generate_name()` method that starts with `.` and stops when it produces another `.`

```
Embedding Table (27 × 27):

         a    b    c  ...  z    .
  .   [ 0.1  0.3  0.2 ... 0.0  0.0 ]   ← "After '.', what comes next?"
  a   [ 0.0  0.2  0.1 ... 0.1  0.3 ]   ← "After 'a', what comes next?"
  b   [ 0.2  0.0  0.1 ... 0.0  0.1 ]
  ...
```

In [ ]:
class BigramNameModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, x, y=None):
        """
        x: (B,) tensor of character indices
        y: (B,) tensor of target character indices (optional)
        Returns: logits (B, vocab_size), loss (scalar or None)
        """
        logits = self.token_embedding_table(x)   # (B, C) where C = vocab_size

        if y is None:
            loss = None
        else:
            loss = F.cross_entropy(logits, y)

        return logits, loss

    def generate_name(self, max_len=20):
        """
        Generate a single name by sampling character by character.
        Starts with '.' (index 0), stops when '.' is produced again.
        """
        idx = torch.tensor([stoi['.']], device=device)  # start with '.'
        name_chars = []

        for _ in range(max_len):
            logits, _ = self(idx)
            probs = F.softmax(logits, dim=-1)           # (1, C)
            idx = torch.multinomial(probs, num_samples=1).squeeze()  # scalar
            ch = itos[idx.item()]
            if ch == '.':
                break  # end of name
            name_chars.append(ch)
            idx = idx.unsqueeze(0)  # reshape for next iteration

        return ''.join(name_chars)

In [ ]:
# Create the model and move to device
model = BigramNameModel(vocab_size).to(device)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Model has {num_params} parameters")
print(f"(That's a {vocab_size}×{vocab_size} = {vocab_size*vocab_size} embedding table)")

### Test Before Training

Let's see what the untrained model generates — it should be complete gibberish:

In [ ]:
print("Names from UNTRAINED model:")
print("-" * 30)
for i in range(10):
    print(f"  {model.generate_name()}")

As expected — random sequences of characters! Let's fix that by training.

---
## Section 8: Training the Model

Our training loop follows the same structure from Day 7 Part 2:

1. **Sample** a mini-batch of bigram pairs
2. **Forward pass**: compute logits and cross-entropy loss
3. **Backward pass**: compute gradients
4. **Update**: adjust the embedding weights using AdamW
5. **Evaluate**: every `eval_interval` steps, print stable train/val loss

We also record the loss history so we can plot training curves afterwards.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Track loss history for plotting
train_losses = []
val_losses = []
steps_recorded = []

for step in range(max_iters):
    
    # Periodic evaluation
    if step % eval_interval == 0 or step == max_iters - 1:
        losses = estimate_loss()
        train_losses.append(losses['train'].item())
        val_losses.append(losses['val'].item())
        steps_recorded.append(step)
        print(f"Step {step:5d}: train loss = {losses['train']:.4f}, val loss = {losses['val']:.4f}")
    
    # Sample a batch
    xb, yb = get_batch('train')
    
    # Forward pass
    logits, loss = model(xb, yb)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()
    
    # Zero gradients
    optimizer.zero_grad(set_to_none=True)

print("\nTraining complete! ✅")

### Training Curves

Let's visualise how the loss decreased during training:

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(steps_recorded, train_losses, label='Train Loss', marker='o', color='#1E88E5')
plt.plot(steps_recorded, val_losses, label='Val Loss', marker='s', color='#E53935')
plt.xlabel('Training Step')
plt.ylabel('Cross-Entropy Loss')
plt.title('Training & Validation Loss Over Time')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nFinal train loss: {train_losses[-1]:.4f}")
print(f"Final val loss:   {val_losses[-1]:.4f}")

**How good is this loss?**

The theoretical **minimum loss** for a perfect bigram model depends on the true bigram statistics in the data. With 27 tokens, a uniform random model would have loss $= -\ln(1/27) \approx 3.30$. Any loss well below that means the model has learned something meaningful!

---
## Section 8: Generating Names 🎉

The moment of truth! Let's generate some names from our trained model.

**How Generation Works:**
1. Start with the boundary token `.` (index 0)
2. The model predicts probabilities for the next character
3. We **sample** from this distribution (not just pick the highest — this adds variety!)
4. Repeat until the model produces another `.` (end of name) or we hit `max_len`

In [ ]:
print("=" * 40)
print("  Generated Names (Trained Model)")
print("=" * 40)
for i in range(20):
    name = model.generate_name(max_len=20)
    print(f"  {i+1:2d}. {name}")
print("=" * 40)

**Observations:**
- The names look somewhat plausible — they follow common character patterns in English names
- Some might even sound like real names!
- However, the model only looks at the **previous character** — so it can't capture longer patterns like "the" or "tion"
- A more powerful model (e.g., an MLP or Transformer) would produce better names

---
## Section 9: Visualising the Learned Bigram Probabilities

Let's peek inside the model and see what it learned! The embedding table is essentially a **27 × 27 matrix** where entry `(i, j)` tells us:

> "After character `i`, how likely is character `j` to come next?"

We convert the raw logits to probabilities using softmax and display them as a heatmap.

In [ ]:
# Extract the learned embedding table and convert to probabilities
with torch.no_grad():
    W = model.token_embedding_table.weight.cpu()  # (27, 27) raw logits
    P = torch.softmax(W, dim=1)                   # (27, 27) probabilities

# Create the heatmap
fig, ax = plt.subplots(figsize=(14, 14))
im = ax.imshow(P.numpy(), cmap='Blues')

# Label axes with characters
labels = [itos[i] for i in range(vocab_size)]
ax.set_xticks(range(vocab_size))
ax.set_xticklabels(labels, fontsize=10)
ax.set_yticks(range(vocab_size))
ax.set_yticklabels(labels, fontsize=10)

ax.set_xlabel('Next Character (predicted)', fontsize=12)
ax.set_ylabel('Current Character (input)', fontsize=12)
ax.set_title('Learned Bigram Probabilities', fontsize=14)

# Annotate each cell with the probability value
for i in range(vocab_size):
    for j in range(vocab_size):
        val = P[i, j].item()
        if val > 0.02:  # only annotate non-trivial probabilities
            color = 'white' if val > 0.15 else 'black'
            ax.text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=6, color=color)

plt.colorbar(im, ax=ax, label='Probability', shrink=0.8)
plt.tight_layout()
plt.show()

### Reading the Heatmap

- **Row `.`** (the boundary token): shows which characters most commonly **start** a name. You should see peaks at common starting letters like `a`, `j`, `m`, `s`.
- **Column `.`**: shows which characters most commonly **end** a name. You should see peaks at common ending letters like `a`, `n`, `e`, `y`.
- **Diagonal patterns**: letters that commonly double up (e.g., `l` → `l`, `n` → `n`).

In [ ]:
# Top 5 most likely starting characters
start_probs = P[stoi['.'], :]
top_starts = torch.topk(start_probs, 5)
print("Top 5 starting characters:")
for prob, idx in zip(top_starts.values, top_starts.indices):
    print(f"  '{itos[idx.item()]}' — {prob.item():.1%}")

print()

# Top 5 most likely ending characters (characters most likely followed by '.')
end_probs = P[:, stoi['.']]
top_ends = torch.topk(end_probs, 5)
print("Top 5 ending characters:")
for prob, idx in zip(top_ends.values, top_ends.indices):
    if itos[idx.item()] != '.':
        print(f"  '{itos[idx.item()]}' — {prob.item():.1%}")

---
## Section 10: Comparing with Counting-Based Bigram

As a sanity check, let's build a bigram model the "classical" way — just counting character pairs — and compare it to our neural network approach.

In [ ]:
# Count bigrams from the dataset
N = torch.zeros((vocab_size, vocab_size), dtype=torch.int32)

for name in names:
    chars_in_name = ['.'] + list(name) + ['.']
    for ch1, ch2 in zip(chars_in_name, chars_in_name[1:]):
        N[stoi[ch1], stoi[ch2]] += 1

# Convert counts to probabilities (add smoothing to avoid log(0))
P_count = (N + 1).float()
P_count = P_count / P_count.sum(dim=1, keepdim=True)

# Compute negative log-likelihood on the same data
nll = 0.0
count = 0
for name in names:
    chars_in_name = ['.'] + list(name) + ['.']
    for ch1, ch2 in zip(chars_in_name, chars_in_name[1:]):
        prob = P_count[stoi[ch1], stoi[ch2]]
        nll += -torch.log(prob).item()
        count += 1

print(f"Counting-based bigram average NLL: {nll / count:.4f}")
print(f"Neural bigram final val loss:      {val_losses[-1]:.4f}")
print(f"\nBoth should be close — they're modelling the same statistics!")

---
## Summary & Reflection 📝

### What We Built
- ✅ Downloaded and explored the `names.txt` dataset (32K baby names)
- ✅ Built a character-level tokenizer with `stoi` / `itos` and a boundary token `.`
- ✅ Constructed bigram (input → target) pairs from all names
- ✅ Implemented a `BigramNameModel` using `nn.Embedding`
- ✅ Trained with `CrossEntropyLoss` + `AdamW` optimiser
- ✅ Generated plausible new names by sampling from the trained model
- ✅ Visualised the learned bigram probabilities as a heatmap
- ✅ Verified that our neural approach matches classical counting

### Key Takeaways

| Concept | What We Learned |
|---------|----------------|
| **Boundary token** | Using `.` to mark name start/end lets the model learn which characters begin and end names |
| **Embedding as lookup** | In a bigram model, the embedding table IS the entire model — each row stores next-char logits |
| **Cross-entropy loss** | Measures how well predicted probabilities match the true next character |
| **Neural ≈ Counting** | For bigrams, the neural network converges to the same solution as simple counting |

### Limitations
The bigram model only looks at **one character** at a time. It can't capture:
- Common letter combinations like "th", "ch", "tion"
- Name-level patterns like "names starting with 'Chr' often end with 'is' or 'ina'"

**To go further:** an MLP or Transformer-based model could use longer context and generate much more realistic names!